## XML Schema for gmso ForceField

We use XML standard 1.0 and enforce strict validation (no extra attributes or new patterns allowed) via `lxml` and custom logic to create a forcefield schema with the following major tags:

1. Metadata tag **`<FFMetadata/>`**: This tag can be used to define default units for the forcefield schema. We can also save the `electrostaticScale` as well as `nonBonded14Scale` for the forcefield in the metadata tag.
2. Potential Group Tags **`<AtomTypes/>`**, **`<BondTypes/>`**, **`<AngleTypes/>`**, **`<DihedralTypes/>`**. These tags can be used to group togehter potentials with similar attributes like expression and parameters. They consolidate information regarding individual poetentials. A ForceField XML can have multiple group tags of the same kind.
3. Individual Potentials **`<AtomType/>`**, **`<BondType/>`**, **`<AngleType/>`**, **`<DihedralType/>`** and **`<ImproperType/>`** contains attributes and subtags to represent individual potentials.

The snippet below shows the rules for representing an **`<AtomType>`** element.

```xsd
 <xsd:complexType name="AtomType">
        <xsd:sequence>
            <xsd:element ref="Parameters" minOccurs="0" maxOccurs="1"/>
        </xsd:sequence>
        <xsd:attribute name="name" type="xsd:string" use="optional"/>
        <xsd:attribute name="mass" type="xsd:float" use="optional"/>
        <xsd:attribute name="element" type="xsd:string" use="optional"/>
        <xsd:attribute name="charge" type="xsd:float" use="optional"/>
        <xsd:attribute name="expression" type="xsd:string" use="optional"/>
        <xsd:attribute name="independent_variables" type="xsd:string" use="optional"/>
        <xsd:attribute name="atomclass" type="xsd:string" use="optional"/>
        <xsd:attribute name="doi" type="xsd:string" use="optional"/>
        <xsd:attribute name="overrides" type="xsd:string" use="optional"/>
        <xsd:attribute name="definition" type="xsd:string" use="optional"/>
        <xsd:attribute name="description" type="xsd:string" use="optional"/>
    </xsd:complexType>
```

As shown above, when creating a `ForceField` object from the serialized representation of a `ForceField`, the strings for expression are converted into symbols using the `sympy` package and units are handled by yt's `unyt` package.

An example forcefield XML (`tip3p.xml`) following the schema above is shown below

```xml
<?xml version='1.0' encoding='UTF-8'?>
<ForceField name="TIP3P" version="0.0.1"> 
  <!-- Defines units as metadata-->
  <FFMetaData>
    <Units energy="kJ/mol" mass="amu" charge="elementary_charge" distance="nm"/>
  </FFMetaData>
  <!-- Potentials can be grouped together by expression and can have optional names -->
  <AtomTypes expression="4*epsilon * ((sigma/r)**12 - (sigma/r)**6)">
     <!--   Units for parameters are defined in this tag    -->
    <ParametersUnitDef parameter="epsilon" unit="kJ/mol"/>
    <ParametersUnitDef parameter="sigma" unit="nm"/>
    <AtomType name="opls_111" element="O" charge="-0.834" mass="16" definition="O" description="water O">
      <Parameters>
        <Parameter name="epsilon" value="0.636386"/>
        <Parameter name="sigma" value="0.315061"/>
      </Parameters>
    </AtomType>
    <AtomType name="opls_112" element="H" charge="0.417" mass="1.011" definition="H">
      <Parameters>
        <Parameter name="epsilon" value="0.0"/>
        <Parameter name="sigma" value="1.0"/>
      </Parameters>
    </AtomType>
  </AtomTypes>
  <BondTypes expression="0.5 * k * (r-r_eq)**2">
    <ParametersUnitDef parameter="k" unit="kJ/mol/nm**2"/>
    <ParametersUnitDef parameter="r_eq" unit="nm"/>
    <BondType name="BondType-Harmonic-1" type1="opls_111" type2="opls_112">
      <Parameters>
        <Parameter name="k" value="502416.0"/>
        <Parameter name="r_eq" value="0.09572"/>
      </Parameters>
    </BondType>
  </BondTypes>
  <AngleTypes expression="0.5 * k * (theta - theta_eq)**2">
    <ParametersUnitDef parameter="k" unit="kJ/(mol*radian**2)"/>
    <ParametersUnitDef parameter="theta_eq" unit="radian"/>
    <AngleType name="AngleType-Harmonic-1" type1="opls_112" type2="opls_111" type3="opls_112">
      <Parameters>
        <Parameter name="k" value="682.02"/>
        <Parameter name="theta_eq" value="1.824218134"/>
      </Parameters>
    </AngleType>
  </AngleTypes>
</ForceField>
```